In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet

import warnings; warnings.simplefilter('ignore')

# Clean Revenue and Budget

In [2]:
md = pd. read_csv('../data/movies_metadata.csv')

In [3]:
md = (md.drop(['budget'], axis=1)
         .join(md['budget'].apply(pd.to_numeric, errors='coerce')))

md = md[md['budget'].notnull()]

In [4]:
md = md.replace(0,'NaN')
md = md[md.budget.notnull()]
md = md[md.budget != 'NaN']
md.loc[:, 'budget'] = pd.to_numeric(md['budget'])
md = md[md['budget'] != 0]
md = md[md['revenue'].notnull()]
md = md[md['revenue'] != 'NaN']

# Clean Genres and Production Companies

In [5]:
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [6]:
md['production_companies'] = md['production_companies'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

## Add Language Features as Integer

In [7]:
md["Eng"] = md["original_language"] == 'en'
md["Eng"] = md["Eng"].astype(int)

In [8]:
md.head()

,adult,belongs_to_collection,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,budget,Eng
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...","[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,...,81,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,NaN,7.7,5415,30000000.0,1
1,False,NaN,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,...,104,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,NaN,6.9,2413,65000000.0,1
3,False,NaN,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,...,127,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,NaN,6.1,34,16000000.0,1
5,False,NaN,"[Action, Crime, Drama, Thriller]",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",17.9249,...,170,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,NaN,7.7,1886,60000000.0,1
8,False,NaN,"[Action, Adventure, Thriller]",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,5.23158,...,106,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,NaN,5.5,174,35000000.0,1


In [9]:
md.shape

(5381, 25)

In [10]:
md.to_csv('metadata_only_budget.csv')

In [10]:
md.columns

Index([u'adult', u'belongs_to_collection', u'genres', u'homepage', u'id',
       u'imdb_id', u'original_language', u'original_title', u'overview',
       u'popularity', u'poster_path', u'production_companies',
       u'production_countries', u'release_date', u'revenue', u'runtime',
       u'spoken_languages', u'status', u'tagline', u'title', u'video',
       u'vote_average', u'vote_count', u'budget', u'Eng'],
      dtype='object')